In [ ]:
import pandas as pd
distance = pd.read_json("analysis_5/distance.json")

print(distance)

In [ ]:
import pandas as pd
structure = pd.read_json("analysis_5/structure.json")

print(structure)

In [ ]:

import pandas as pd
assortativity = pd.read_json("analysis_5/assortativity.json")

print(assortativity)

In [ ]:

import pandas as pd
centrality = pd.read_json("analysis_5/centrality.json")

print(centrality)

In [ ]:
import pandas as pd

# Metodo 1: Usando l'index del DataFrame
df = pd.DataFrame({
    'Niche': [453, 4669, 0.0228],
    'Almost-niche': [14892, 439100, 0.0020],
    'Mixed': [15345, 83156, 0.0004]
}, index=['Nodes', 'Edges', 'Density'])  # <-- Nomi delle righe come index

# Converti con index=True (default)
latex_table = df.to_latex(
    caption='Structural properties',
    label='tab:structure',
    float_format="%.4f"
)
print(latex_table)

In [ ]:
import json
import os

import networkx as nx
with open("niche_graph.json") as f:
    graph_dict = json.load(f)

niche_graph = nx.node_link_graph(graph_dict)

with open("almost_niche_graph.json") as f:
    graph_dict = json.load(f)

almost_niche_graph = nx.node_link_graph(graph_dict)

with open("mixed_graph.json") as f:
    graph_dict = json.load(f)

mixed_graph = nx.node_link_graph(graph_dict)

In [ ]:
from networkx.algorithms import bipartite

# 1. Prendi i gradi dei nodi originali (fondamentale per il benchmark)
top_nodes = [n for n, d in mixed_graph.nodes(data=True) if d['bipartite'] == 1]
deg_sequence = [d for n, d in mixed_graph.degree(top_nodes)]

# 2. Genera un grafo bipartito casuale con la stessa densità (Modello Erdos-Renyi)
# n = num_star, m = num_emergenti, p = probabilità di arco (densità)
n = len(top_nodes)
m = mixed_graph.number_of_nodes() - n
p = mixed_graph.number_of_edges() / (n * m)

G_random = bipartite.random_graph(n, m, p)

# 3. Calcola il clustering sul grafo random e confronta
random_clust = bipartite.clustering(G_random)
avg_random_clust = sum(random_clust.values()) / len(random_clust)

print(f"Reale: 0.0238, Casuale: {avg_random_clust}")

In [5]:
import networkx as nx
import networkit as nk
import numpy as np
# 1. Genera il benchmark BIPARTITO reale con NetworkX
n = 453
m = 14892
p = 0.0004
nxbip = nx.bipartite.random_graph(n, m, p)

# 2. Converti in NetworKit
G_rand = nk.nxadapter.nx2nk(nxbip)

# 3. Calcola l'Average Path Length (con campionamento per velocità)
# Invece di EffectiveDiameter, usa l'approssimazione della distanza media
dist_alg = nk.distance.APSP(G_rand)
dist_alg.run()
avg_path_rand = dist_alg.getDistances(asarray=True)
valid_distances = avg_path_rand[avg_path_rand > 0]
valid_distances = valid_distances[valid_distances < G_rand.numberOfNodes()]
average_path_length = np.mean(valid_distances)
print(f"ASP del Benchmark Bipartito: {average_path_length}")

ASP del Benchmark Bipartito: 7.030791838966542


In [ ]:
import pandas as pd
import math
import numpy as np

degree_niche = float(np.mean([d for n, d in niche_graph.degree()]))
degree_almost__niche = float(np.mean([d for n, d in almost_niche_graph.degree()]))
nodes_niche = 453
nodes_almost = 14892
nodes_mixed = 15345


# Metodo 1: Usando l'index del DataFrame
df = pd.DataFrame({
    'Niche': [round(2.56384,4), round(0.118710, 4)],
    'Niche benchmark': [math.log(nodes_niche)/math.log(degree_niche),degree_niche/nodes_niche],
    'Almost-niche': [round(3.098455,4), round(0.245336,4)],
    'Almost-niche benchmark' : [math.log(nodes_almost)/math.log(degree_almost__niche),degree_almost__niche/nodes_almost],
    'Mixed': [round(3.754041,4), round(0.023887,4)],
    'Mixed benchmark':[round(avg_path_rand,4), round(avg_random_clust,4)]
}, index=['Average SP', 'Clustering Coefficient'])  # <-- Nomi delle righe come index

# Converti con index=True (default)
latex_table = df.to_latex(
    caption='Structural properties',
    label='tab:structure',
    float_format="%.4f"
)
print(latex_table)